In [ ]:
import pickle
import yaml
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

SAVE_PATH = Path(os.path.abspath("")).parent / "_static"
RESULTS_PATH = Path(os.path.abspath("")).parent / "results"

In [ ]:
def saveFig(name, fig):
    fig.savefig(
        name,
        dpi=None,
        facecolor=fig.get_facecolor(),
        edgecolor="none",
        orientation="portrait",
        format="png",
        transparent=False,
        bbox_inches="tight",
        pad_inches=0.2,
        metadata=None,
    )

In [ ]:
# Update the path belows to the multirun directories containing the results for FedProx and FedAvg
path_fedprox_resutls = RESULTS_PATH / "FedProx"
path_fedavg_results = RESULTS_PATH / "FedAvg"

In [ ]:
# load all results
def read_pickle_and_config(path_to_pickle):
    with open(path_to_pickle, "rb") as handle:
        data = pickle.load(handle)

    config_path = Path(path_to_pickle).parent / "config.json"
    with open(config_path, "r") as file:
        config = yaml.safe_load(file)

    return data, config


# loads results.pkl and config of each run
def process_data(path_fedprox_results):
    res_list = []
    for results in list(Path(path_fedprox_results).glob("**/*.pkl")):
        data, config = read_pickle_and_config(results)
        data_cen = data["history"].metrics_centralized
        loss_cen = data["history"].losses_centralized
        stragglers = config["algorithm"]["stragglers_fraction"]
        res_list.append(
            {
                "stragglers": stragglers,
                "mu": config["algorithm"]["mu"],
                "accuracy": np.array([d[1] for d in data_cen["accuracy"]]),
                "loss": np.array([d[1] for d in loss_cen]),
            }
        )

    # to DataFrame
    df = pd.DataFrame.from_dict(res_list)
    # Grouping the DataFrame by 'mu' and 'stragglers' columns
    grouped_df = df.groupby(["mu", "stragglers"])

    # Calculating the mean and standard deviation of 'accuracy' and 'loss' columns
    mean_df = grouped_df[["accuracy", "loss"]].mean()
    return mean_df.reset_index()

In [ ]:
fedprox_df = process_data(path_fedprox_resutls)
fedavg_df = process_data(path_fedavg_results)

In [ ]:
# let's add a new column to each dataframe that we'll use as label
fedprox_df["strategy"] = "FedProx"
fedavg_df["strategy"] = "FedAvg"

# Now let's fuse both datframes
df = pd.concat([fedavg_df, fedprox_df])
df = df.reset_index()
df

In [ ]:
def viz(dataframe):
    fig, axs = plt.subplots(figsize=(18, 12), nrows=2, ncols=3)
    for row_idx, col_label in enumerate(["loss", "accuracy"]):
        for col_idx, strag_ratio in enumerate([0.0, 0.5, 0.9]):
            ax = axs[row_idx, col_idx]
            exp_data = dataframe.loc[(dataframe["stragglers"] == strag_ratio)]
            for _, exp in exp_data.iterrows():
                y = exp[col_label]
                rounds_array = np.array(range(len(y)))
                label = exp["strategy"]
                label = f"{label}($\mu$={exp['mu']})" if label == "FedProx" else label
                ax.plot(rounds_array, y, label=label, linewidth=2.0)

            ax.grid()
            ax.legend(fontsize=14)
            ax.set_xlabel("Round", fontsize=14)
            ax.set_ylabel("Loss" if col_idx == 0 else "Accuracy", fontsize=14)

            ax.tick_params(axis="both", which="major", labelsize=14)
            ax.tick_params(axis="both", which="minor", labelsize=14)

            if row_idx == 0:
                ax.set_title(f"{int(strag_ratio*100)}% Stragglers", fontsize=14)
    return fig

In [ ]:
f = viz(df)

In [ ]:
saveFig(SAVE_PATH/"FedProx_mnist.png", f)